## Pytorch sur des séquences de texte

#### Import des librairies

In [87]:
import torchtext # pip install torchtext
import torch
import torch.nn as nn
import torch.optim as optim
#from torchtext.data import Field, TabularDataset, BucketIterator
from torchtext.legacy.data import Field, TabularDataset, BucketIterator

In [88]:
import spacy
spacy_en = spacy.load('en_core_web_trf')

def tokenizer(text): # create a tokenizer function
    return [tok.text for tok in spacy_en.tokenizer(text)]

#### Définition des prétraitements sur le texte

In [182]:
TEXT = Field(sequential = True, lower = True, include_lengths = False,
            pad_token = "<pad>", unk_token = "<unk>",
            batch_first = True, tokenize = tokenizer)

LABELS = Field(sequential=False, use_vocab=False)

#### Tokenization

#### Création des datasets

In [189]:
train_dataset, test_dataset = TabularDataset.splits(
    path="./", format='tsv',
    train='t2.tsv', test='toto.tsv',
    fields=[('text', TEXT), ('labels', LABELS)])

train_dataset[0].labels

TypeError: __init__() got an unexpected keyword argument 'sep'

#### Gestion des batchs

In [162]:
device = torch.device('cuda')
train_iter, test_iter = BucketIterator.splits(
    (train_dataset, test_dataset), batch_size=160,
    sort_key = lambda x: len(x.text), device=device,
    sort_within_batch = True, shuffle = True, repeat=False)


#### Gestion du vocabulaire et des word Embeddings

In [184]:
TEXT.build_vocab(train_dataset, min_freq=2, vectors = 'glove.6B.50d')
next(iter(train_iter))

AttributeError: 'Example' object has no attribute 'labels'

#### Création du modèle

In [164]:
class LSTMModele(nn.Module):
    def __init__(self, embedding_dim=50):
        super(LSTMModele, self).__init__()
        self.embeddings = nn.Embedding.from_pretrained(TEXT.vocab.vectors, freeze = False)# une couche qui ne marche qu'avec les imports qui ne marchent pas...
        self.lstm = nn.LSTM(input_size = embedding_dim, hidden_size = embedding_dim, batch_first=True)
        self.fc = nn.Linear(embedding_dim, 2) # 2 car pos neg
    
    def forward(self, inputs):
        embeds = self.embeddings(inputs) # pour faire le lien entre indice et vecteur du mot associé
        outputs, (h_n,c_n) = self.lstm(embeds)
        x = h_n[0]
        x = self.fc(x)
        return x

In [165]:
net = LSTMModele(embedding_dim = 50).to(device)

In [166]:
criterion = nn.CrossEntropyLoss()
import torch.optim as optim
optimizer = optim.Adam(net.parameters(), lr=0.01)

Puis code de BOUCLE d'APPRENTISSAGE + MESURE DES PERFORMANCES + accuracy_score IDENTIQUES

In [183]:
%%time

nb_epoch = 200
for epoch in range(nb_epoch):
    for batch in train_iter.batches:
        data = batch.text.to(device)
        labels = batch.labels.to(device)
        
        outputs = net(data)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print ('epoch : ' + str(epoch))

print('Finished Training')

AttributeError: 'list' object has no attribute 'text'

#### Mesure des performances

In [139]:
import numpy as np
all_labels = []
all_preds = []

# print(type(train_iter.batches))
# print(type(test_iter.batches))

with torch.no_grad():
    for batch in enumerate(test_iter.batches):
        data = batch.text.to(device)
        labels = batch.labels.to(device)

        outputs = net(data)
        _, predicted = torch.max(outputs.data, 1)

        all_preds.append(predicted.cpu().numpy())
        all_labels.append(labels.cpu().numpy())

all_labels = np.concatenate(all_labels)
all_preds = np.concatenate(all_preds)

ValueError: need at least one array to concatenate

#### Without Word Embeddings

In [119]:
from sklearn.metrics import accuracy_score
accuracy_score(all_labels,all_preds)

C:\Users\Lucas\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
C:\Users\Lucas\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


nan